In [1]:
import numpy as np
import nltk
import string
import pandas as pd
from nltk.stem.snowball import SnowballStemmer

# Use English stemmer.
stemmer = SnowballStemmer("english")

In [2]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
dataset=pd.read_csv('40.csv')
test=pd.read_csv('10.csv')

Creating the vocabulary without stop-words

In [4]:
#Text Pre-Processing
dataset=dataset['text'].str.replace('[{}]'.format(string.punctuation), '')
dataset

dataset=dataset.str.lower().str.split()
#Removing stop words
dataset=dataset.apply(lambda w: [item for item in w if item not in stop])

dataset=dataset.str.join(" ")
Total_Vocabulary = pd.DataFrame(dataset.str.split(expand=True).stack().value_counts())
Total_Vocabulary.reset_index(level=0, inplace=True)
Total_Vocabulary.columns=['word','f']
dataset

0     midst covid19 pandemic eating healthy food rem...
1     specific foods help protect virus nutritious d...
2     may able share meals friends loved ones lots w...
3     eating healthy diet strict limitations staying...
4     healthy eating doesn’t overly complicated feel...
5     need balance protein fat carbohydrates fiber v...
6     protein gives energy get go—and keep going—whi...
7     much protein harmful people kidney disease lat...
8     bad fats wreck diet increase risk certain dise...
9     fact healthy fats—such omega3s—are vital physi...
10    including healthy fat diet help improve mood b...
11    eating foods high dietary fiber grains fruit v...
12    well leading osteoporosis getting enough calci...
13    whatever age gender it’s vital include calcium...
14    carbohydrates one body’s main sources energy c...
15    cutting back white bread pastries starches sug...
16    switching healthy diet doesn’t nothing proposi...
17    don’t perfect don’t completely eliminate f

Estimating the prior probabilities of all "labels", i.e. words in vocabulary

In [5]:
prior_p=[]
word_Sum=0
for i in range(len(Total_Vocabulary)):
  word_Sum+=Total_Vocabulary['f'][i]
for i in range(len(Total_Vocabulary)):
  prior_p.append(Total_Vocabulary['f'][i]/word_Sum)
Total_Vocabulary["prior_P"]=prior_p
Total_Vocabulary

,word,f,prior_P
0,healthy,14,0.024263
1,eating,12,0.020797
2,diet,11,0.019064
3,food,10,0.017331
4,foods,9,0.015598
5,help,9,0.015598
6,energy,6,0.010399
7,fats,6,0.010399
8,also,5,0.008666
9,sugar,5,0.008666


In [6]:
word_Sum

577

In [7]:
combo =[Total_Vocabulary['word'][0],Total_Vocabulary['word'][1]]
tog= dataset.apply(lambda sentence: all(word in sentence for word in combo))
c=0
for h in tog:
  if h==True:
    c+=1
print(tog,c)

0      True
1     False
2     False
3      True
4      True
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32     True
33    False
34    False
35    False
36     True
37    False
38    False
39    False
Name: text, dtype: bool 5


Estimating the class-conditional probabilities of all words

In [8]:
n=10**(-30)
for i in range(len(Total_Vocabulary)):
    cond_p=[]
    for j in range(len(Total_Vocabulary)):
        if i == j :
            cond_p.append(0.99999)
        else: 
            combo =[Total_Vocabulary['word'][i],Total_Vocabulary['word'][j]]
            tog= dataset.apply(lambda sentence: all(word in sentence for word in combo))
            c=0
            for h in tog:
              if h==True:
                c+=1
            if c == 0 :
                cond_p.append(n)
            else:
                cond_p.append(c/Total_Vocabulary['f'][i])
    Total_Vocabulary[i]= cond_p
# each column gives conditional probability p(x2/x1=i)
Total_Vocabulary

,word,f,prior_P,0,1,2,3,4,5,6,...,367,368,369,370,371,372,373,374,375,376
0,healthy,14,0.024263,9.999900e-01,4.166667e-01,6.363636e-01,6.000000e-01,4.444444e-01,2.222222e-01,1.666667e-01,...,1.000000e-30,1.000000e-30,1.000000e+00,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30
1,eating,12,0.020797,3.571429e-01,9.999900e-01,2.727273e-01,9.000000e-01,5.555556e-01,3.333333e-01,1.666667e-01,...,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e+00,1.000000e-30,1.000000e-30,1.000000e+00
2,diet,11,0.019064,5.000000e-01,2.500000e-01,9.999900e-01,5.000000e-01,5.555556e-01,4.444444e-01,1.666667e-01,...,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e+00,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e+00
3,food,10,0.017331,4.285714e-01,7.500000e-01,4.545455e-01,9.999900e-01,1.000000e+00,6.666667e-01,3.333333e-01,...,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e+00,1.000000e-30,1.000000e-30,1.000000e+00
4,foods,9,0.015598,2.857143e-01,4.166667e-01,4.545455e-01,9.000000e-01,9.999900e-01,4.444444e-01,1.666667e-01,...,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e+00
5,help,9,0.015598,1.428571e-01,2.500000e-01,3.636364e-01,6.000000e-01,4.444444e-01,9.999900e-01,1.666667e-01,...,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e+00
6,energy,6,0.010399,7.142857e-02,8.333333e-02,9.090909e-02,2.000000e-01,1.111111e-01,1.111111e-01,9.999900e-01,...,1.000000e+00,1.000000e+00,1.000000e-30,1.000000e-30,1.000000e+00,1.000000e-30,1.000000e-30,1.000000e+00,1.000000e-30,1.000000e-30
7,fats,6,0.010399,2.857143e-01,1.000000e-30,9.090909e-02,2.000000e-01,1.111111e-01,1.000000e-30,1.000000e-30,...,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e+00,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30
8,also,5,0.008666,1.000000e-30,1.666667e-01,1.818182e-01,3.000000e-01,1.111111e-01,2.222222e-01,1.666667e-01,...,1.000000e+00,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e+00,1.000000e-30,1.000000e-30,1.000000e+00
9,sugar,5,0.008666,1.428571e-01,1.000000e-30,1.000000e-30,2.000000e-01,1.111111e-01,1.000000e-30,3.333333e-01,...,1.000000e-30,1.000000e+00,1.000000e-30,1.000000e-30,1.000000e+00,1.000000e-30,1.000000e-30,1.000000e+00,1.000000e-30,1.000000e-30


In [9]:
test=pd.read_csv('10.csv')

In [10]:
#Text Pre-Processing
t=test['text'].str.replace('[{}]'.format(string.punctuation), '')

t=t.str.lower().str.split()
#Removing stop words
t=t.apply(lambda w: [item for item in w if item not in stop])
t[0]

['eating', 'healthy', 'food', 'important', 'maintainng', 'good']

In each test sentence, calculating the most likely word in the missing position along with probability

In [11]:
arr=[]

for i in range(len(t)):
  post_p=[]
  for k in range(len(Total_Vocabulary)):
    post_p.append(Total_Vocabulary['prior_P'][k])
    for j in range(len(t[i])): 
      for h in range(len(Total_Vocabulary)):
        if t[i][j] ==Total_Vocabulary['word'][h]:
          if h!=k:
            post_p[k]*= Total_Vocabulary[h][k]
          break
  arr.append(post_p)  

In [12]:
Post=pd.Series(arr)
Post

0    [6.0658578856152516e-33, 2.2282743253280516e-3...
1    [0.01544036552702064, 0.0010128519660582052, 0...
2    [3.2167428181293e-33, 2.0257039321164114e-153,...
3    [5.1467885090068795e-33, 5.67197100992595e-63,...
4    [6.551126516464472e-63, 1.1230502599653383e-62...
5    [2.0219526285384175e-93, 3.4662045060658593e-9...
6    [1.0109763142692088e-122, 1.0398613518197578e-...
7    [7.800743165657476e-64, 3.094825451844516e-63,...
8    [4.38791803068233e-94, 1.0398613518197576e-32,...
9    [2.2746967071057198e-63, 9.35875216637782e-93,...
dtype: object

In [13]:
Pred=[]
m=np.zeros(len(t))
for i in range(len(t)):
  
  k=0
  for j in range(len(Post[0])):
    if m[i]<Post[i][j]:
      k=j
      m[i]=Post[i][j]
  Pred.append(Total_Vocabulary['word'][k])
Pred      
  



['diet',
 'healthy',
 'able',
 'diet',
 'food',
 'energy',
 'k',
 'food',
 'eat',
 'food']

In [14]:
test= pd.concat([test, pd.Series(Pred, index=test.index, name='Pred_word')], axis=1)
test

,text,Pred_word
0,Eating healthy food is important for maintainn...,diet
1,Following a healthy diet will boost your _____,healthy
2,"Avoid eating chemical additives, added sugars ...",able
3,"Your diet should be rich of vitamins D,K, calc...",diet
4,"The healthier the food you eat, the better you...",food
5,"Dehydration causes tiredness, low energy and h...",energy
6,People with kidney disease should avoid eating...,k
7,We should avoid eating transfats. Eating healt...,food
8,Mindless eating is often caused by eating alon...,eat
9,Eating more junk food will make you feel uncom...,food
